# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\\$645 - 675) underperformed compared to schools with smaller budgets (\\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

**Note:**
Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [105]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import sqlite3
import os


current_directory = os.getcwd()
print(current_directory)

# loading data from python.db database
con = sqlite3.connect(r'./data/python.db')

# import 'school' and 'student' tables into pandas dataframe
school_data = pd.read_sql('SELECT * FROM school', con)
student_data = pd.read_sql('SELECT * FROM student', con)


/Users/shaunkumar/Desktop/DS311/Python-HW-Repository/Lab_Assignment


In [2]:
# join the two tables into a single dataframe
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,id_x,Student ID,student_name,gender,grade,school_name,reading_score,math_score,id_y,School ID,type,size,budget
0,0,0,Paul Bradley,M,9th,Huang High School,66,79,0,0,District,2917,1910635
1,1,1,Victor Smith,M,12th,Huang High School,94,61,0,0,District,2917,1910635
2,2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,0,District,2917,1910635
3,3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,0,District,2917,1910635
4,4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [6]:
# Create a District Summary
district_summary = school_data_complete.groupby('type').agg(
    student_count=('Student ID', 'count'),              # Count of students
    school_count=('School ID', 'nunique'),              # Count of unique schools
    total_budget=('budget', 'sum'),
    average_math_score=('math_score', 'mean'),
    average_reading_score=('reading_score', 'mean'),
)
district_summary['overall_average_score'] = (district_summary['average_math_score'] + district_summary['average_reading_score']) / 2

passing_students_math = school_data_complete[school_data_complete['math_score'] >= 70]
passing_students_math_count = passing_students_math.groupby('type')['Student ID'].count()
district_summary['passing_percentage_math'] = (passing_students_math_count / district_summary['student_count']) * 100

passing_students_reading = school_data_complete[school_data_complete['reading_score'] >= 70]
passing_students_reading_count = passing_students_reading.groupby('type')['Student ID'].count()
district_summary['passing_percentage_reading'] = (passing_students_reading_count / district_summary['student_count']) * 100

print(district_summary)

          student_count  school_count  total_budget  average_math_score  \
type                                                                      
Charter           12194             8   12493275585           83.406183   
District          26976             7   70439053973           76.987026   

          average_reading_score  overall_average_score  \
type                                                     
Charter               83.902821              83.654502   
District              80.962485              78.974755   

          passing_percentage_math  passing_percentage_reading  
type                                                           
Charter                 93.701821                   96.645891  
District                66.518387                   80.905249  


In [8]:
# Total number of schools
total_schools = school_data['School ID'].nunique()  

# Print the result
print(f"Total number of schools is: {total_schools}")



Total number of schools is: 15


In [10]:
# Total number of students
total_students = student_data['Student ID'].nunique()  

# Print the result
print(f"Total number of schools is: {total_students}")


Total number of schools is: 39170


In [12]:
# Total budget
total_budget = school_data['budget'].sum()  

# Print the result
print(f"Total budget of the district is: ${total_budget}")



Total budget of the district is: $24649428


In [14]:
# Average math score
avg_math_score = student_data['math_score'].mean()  

# Print the result
print(f"Average Math Score of the district is: {avg_math_score}")




Average Math Score of the district is: 78.98537145774827


In [16]:
# Average reading score
avg_reading_score = student_data['reading_score'].mean()  

# Print the result
print(f"Average Reading Score of the district is: {avg_reading_score}")



Average Reading Score of the district is: 81.87784018381414


In [18]:
# Overall average score
overall_avg_score = (avg_math_score + avg_reading_score) / 2

# Print the result
print(f"Overall Average Score of the district is: {overall_avg_score}")



Overall Average Score of the district is: 80.43160582078121


In [25]:
# Percentage of passing math (70 or greater)
# Count the number of students who have a math score of 70 or greater
passing_students_math = student_data[student_data['math_score'] >= 70]

# Calculate the number of passing students
num_passing_math_students = passing_students_math.shape[0]

# Calculate the total number of students
total_students = student_data.shape[0]

#Calculate the percentage of passing students
passing_math_percentage = (num_passing_math_students / total_students) * 100

# Print the result
print(f"Percentage of Students Passing Math in the District is: {passing_math_percentage}%")

Percentage of Students Passing Math in the District is: 74.9808526933878%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

### Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [67]:
#  Sort and display the top five schools in overall passing rate
# Calculate the passing percentage for math and reading for each student
student_data['passing_math'] = student_data['math_score'] >= 70
student_data['passing_reading'] = student_data['reading_score'] >= 70

# Calculate overall passing rate by combining math and reading passing rates
student_data['overall_passing'] = (student_data['passing_math'] & student_data['passing_reading'])

# Group by 'school_name' and calculate the overall passing percentage for each school
school_passing_rates = student_data.groupby('school_name').agg(
    total_students=('Student ID', 'count'),
    passing_students=('overall_passing', 'sum')
)

# Calculate the overall passing percentage for each school
school_passing_rates['passing_percentage'] = (school_passing_rates['passing_students'] / school_passing_rates['total_students']) * 100

# Sort the schools by passing percentage in descending order and display the top 5
top_5_schools = school_passing_rates.sort_values(by='passing_percentage', ascending=False).head(5)

# Print the top 5 schools based on overall passing rate
display(top_5_schools[['passing_percentage']])


,passing_percentage
school_name,
Cabrera High School,91.334769
Thomas High School,90.948012
Griffin High School,90.599455
Wilson High School,90.582567
Pena High School,90.540541


In [73]:
# Calculate the total budget for each school
total_school_budgets = school_data.groupby('school_name')['budget'].sum()

# Print the total budgets for each school
print(total_school_budgets)


school_name
Bailey High School       3124928
Cabrera High School      1081356
Figueroa High School     1884411
Ford High School         1763916
Griffin High School       917500
Hernandez High School    3022020
Holden High School        248087
Huang High School        1910635
Johnson High School      3094650
Pena High School          585858
Rodriguez High School    2547363
Shelton High School      1056600
Thomas High School       1043130
Wilson High School       1319574
Wright High School       1049400
Name: budget, dtype: int64


In [77]:
# Calculate per student budget

# Calculate the total number of students in each school
students_per_school = school_data.groupby('school_name')['size'].sum()

# Calculate the per student budget for each school
per_student_budget = total_school_budgets / students_per_school

# Print the per student budget for each school
print(per_student_budget)




school_name
Bailey High School       628.0
Cabrera High School      582.0
Figueroa High School     639.0
Ford High School         644.0
Griffin High School      625.0
Hernandez High School    652.0
Holden High School       581.0
Huang High School        655.0
Johnson High School      650.0
Pena High School         609.0
Rodriguez High School    637.0
Shelton High School      600.0
Thomas High School       638.0
Wilson High School       578.0
Wright High School       583.0
dtype: float64


In [209]:
# Cacluate the avg math and reading score
# Calculate the average math and average reading scores for each school
average_scores = student_data.groupby('school_name').agg(
    average_math_score=('math_score', 'mean'),
    average_reading_score=('reading_score', 'mean')
)

# Print the average scores for each school
display(average_scores)




,average_math_score,average_reading_score
school_name,,
Bailey High School,77.048432,81.033963
Cabrera High School,83.061895,83.975780
Figueroa High School,76.711767,81.158020
Ford High School,77.102592,80.746258
Griffin High School,83.351499,83.816757
Hernandez High School,77.289752,80.934412
Holden High School,83.803279,83.814988
Huang High School,76.629414,81.182722
Johnson High School,77.072464,80.966394


#### Find the passing rate for math and reading (above 70 points)

In [83]:
# Find the total counts of math result
math_score_counts = student_data.groupby('school_name')['math_score'].count()

# Find the counts for math result in each school that pass 70 or higher
passing_math = student_data[student_data['math_score'] >= 70]
passing_math_counts = passing_math.groupby('school_name')['math_score'].count()

# Calculate the math passing rate
math_passing_rate = (passing_math_counts / math_score_counts) * 100

print(math_passing_rate)

school_name
Bailey High School       66.680064
Cabrera High School      94.133477
Figueroa High School     65.988471
Ford High School         68.309602
Griffin High School      93.392371
Hernandez High School    66.752967
Holden High School       92.505855
Huang High School        65.683922
Johnson High School      66.057551
Pena High School         94.594595
Rodriguez High School    66.366592
Shelton High School      93.867121
Thomas High School       93.272171
Wilson High School       93.867718
Wright High School       93.333333
Name: math_score, dtype: float64


In [87]:
# Find the total counts of read result
reading_score_counts = student_data.groupby('school_name')['reading_score'].count()


# Find the counts for read result in each school that pass 70 or higher
passing_reading = student_data[student_data['reading_score'] >= 70]
passing_reading_counts = passing_reading.groupby('school_name')['reading_score'].count()

# Calculate the read passing rate
reading_passing_rate = (passing_reading_counts / reading_score_counts) * 100

print(reading_passing_rate)


school_name
Bailey High School       81.933280
Cabrera High School      97.039828
Figueroa High School     80.739234
Ford High School         79.299014
Griffin High School      97.138965
Hernandez High School    80.862999
Holden High School       96.252927
Huang High School        81.316421
Johnson High School      81.222432
Pena High School         95.945946
Rodriguez High School    80.220055
Shelton High School      95.854628
Thomas High School       97.308869
Wilson High School       96.539641
Wright High School       96.611111
Name: reading_score, dtype: float64


In [103]:
# Calculate the overall passing rate (average of the math and reading passing rate)
overall_passing_rate = (math_passing_rate + reading_passing_rate) / 2
print(overall_passing_rate)


school_name
Bailey High School       74.306672
Cabrera High School      95.586652
Figueroa High School     73.363852
Ford High School         73.804308
Griffin High School      95.265668
Hernandez High School    73.807983
Holden High School       94.379391
Huang High School        73.500171
Johnson High School      73.639992
Pena High School         95.270270
Rodriguez High School    73.293323
Shelton High School      94.860875
Thomas High School       95.290520
Wilson High School       95.203679
Wright High School       94.972222
dtype: float64


### Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [97]:
#  Sort and display the worst five schools in overall passing rate
worst_five_schools = overall_passing_rate.sort_values(ascending=True).head(5)
print("Worst Five Schools in Overall Passing Rate:")
print(worst_five_schools)


Worst Five Schools in Overall Passing Rate:
school_name
Rodriguez High School    73.293323
Figueroa High School     73.363852
Huang High School        73.500171
Johnson High School      73.639992
Ford High School         73.804308
dtype: float64


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [135]:
# Create table that lists the average math score for each school of each grade level.
average_math_by_grade = student_data.groupby(['school_name', 'grade'])['math_score'].mean().unstack()

# Reorder columns to ensure they are sorted from 9th to 12th
average_math_by_grade = average_math_by_grade[['9th', '10th', '11th', '12th']]

# Printing results to verify 
print("Average Math Score by Grade Level for Each School:")
display(average_math_by_grade)



Average Math Score by Grade Level for Each School:


grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [129]:
# Calculate the average math score for 9th grade in each school
ninth_grade_students = student_data[student_data['grade'] == '9th']
average_math_9th_grade = ninth_grade_students.groupby('school_name')['math_score'].mean()

# Print
print("Average Math Score for 9th Grade in Each School:")
print(average_math_9th_grade)


Average Math Score for 9th Grade in Each School:
school_name
Bailey High School       77.083676
Cabrera High School      83.094697
Figueroa High School     76.403037
Ford High School         77.361345
Griffin High School      82.044010
Hernandez High School    77.438495
Holden High School       83.787402
Huang High School        77.027251
Johnson High School      77.187857
Pena High School         83.625455
Rodriguez High School    76.859966
Shelton High School      83.420755
Thomas High School       83.590022
Wilson High School       83.085578
Wright High School       83.264706
Name: math_score, dtype: float64


In [111]:
# Calculate the average math score for 10th grade in each school
tenth_grade_students = student_data[student_data['grade'] == '10th']
average_math_10th_grade = tenth_grade_students.groupby('school_name')['math_score'].mean()

# Print
print("Average Math Score for 10th Grade in Each School:")
print(average_math_10th_grade)


Average Math Score for 10th Grade in Each School:
school_name
Bailey High School       76.996772
Cabrera High School      83.154506
Figueroa High School     76.539974
Ford High School         77.672316
Griffin High School      84.229064
Hernandez High School    77.337408
Holden High School       83.429825
Huang High School        75.908735
Johnson High School      76.691117
Pena High School         83.372000
Rodriguez High School    76.612500
Shelton High School      82.917411
Thomas High School       83.087886
Wilson High School       83.724422
Wright High School       84.010288
Name: math_score, dtype: float64


In [117]:
# Calculate the average math score for 11th grade in each school
eleventh_grade_students = student_data[student_data['grade'] == '11th']
average_math_11th_grade = eleventh_grade_students.groupby('school_name')['math_score'].mean()

# Print
print("Average Math Score for 11th Grade in Each School:")
print(average_math_11th_grade)


Average Math Score for 11th Grade in Each School:
school_name
Bailey High School       77.515588
Cabrera High School      82.765560
Figueroa High School     76.884344
Ford High School         76.918058
Griffin High School      83.842105
Hernandez High School    77.136029
Holden High School       85.000000
Huang High School        76.446602
Johnson High School      77.491653
Pena High School         84.328125
Rodriguez High School    76.395626
Shelton High School      83.383495
Thomas High School       83.498795
Wilson High School       83.195326
Wright High School       83.836782
Name: math_score, dtype: float64


In [119]:
# Calculate the average math score for 12th grade in each school
twelfth_grade_students = student_data[student_data['grade'] == '12th']
average_math_12th_grade = twelfth_grade_students.groupby('school_name')['math_score'].mean()

# Print
print("Average Math Score for 12th Grade in Each School:")
print(average_math_12th_grade)


Average Math Score for 12th Grade in Each School:
school_name
Bailey High School       76.492218
Cabrera High School      83.277487
Figueroa High School     77.151369
Ford High School         76.179963
Griffin High School      83.356164
Hernandez High School    77.186567
Holden High School       82.855422
Huang High School        77.225641
Johnson High School      76.863248
Pena High School         84.121547
Rodriguez High School    77.690748
Shelton High School      83.778976
Thomas High School       83.497041
Wilson High School       83.035794
Wright High School       83.644986
Name: math_score, dtype: float64


### Reading Score by Grade 

* Perform the same operations as above for reading scores

In [139]:
# Create table that lists the average reading score for each school of each grade level.
average_reading_by_grade = student_data.groupby(['school_name', 'grade'])['reading_score'].mean().unstack()

# Reorder columns to ensure they are sorted from 9th to 12th
average_reading_by_grade = average_reading_by_grade[['9th', '10th', '11th', '12th']]

# Printing results to verify 
print("Average Reading Score by Grade Level for Each School:")
display(average_reading_by_grade)


Average Reading Score by Grade Level for Each School:


grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [137]:
# Calculate the average reading score for 9th grade in each school
average_reading_9th_grade = ninth_grade_students.groupby('school_name')['reading_score'].mean()

# Print
print("Average Reading Score for 9th Grade in Each School:")
print(average_reading_9th_grade)


Average Reading Score for 9th Grade in Each School:
school_name
Bailey High School       81.303155
Cabrera High School      83.676136
Figueroa High School     81.198598
Ford High School         80.632653
Griffin High School      83.369193
Hernandez High School    80.866860
Holden High School       83.677165
Huang High School        81.290284
Johnson High School      81.260714
Pena High School         83.807273
Rodriguez High School    80.993127
Shelton High School      84.122642
Thomas High School       83.728850
Wilson High School       83.939778
Wright High School       83.833333
Name: reading_score, dtype: float64


In [141]:
# Calculate the average reading score for 10th grade in each school
average_reading_10th_grade = tenth_grade_students.groupby('school_name')['reading_score'].mean()

# Print
print("Average Reading Score for 10th Grade in Each School:")
print(average_reading_10th_grade)


Average Reading Score for 10th Grade in Each School:
school_name
Bailey High School       80.907183
Cabrera High School      84.253219
Figueroa High School     81.408912
Ford High School         81.262712
Griffin High School      83.706897
Hernandez High School    80.660147
Holden High School       83.324561
Huang High School        81.512386
Johnson High School      80.773431
Pena High School         83.612000
Rodriguez High School    80.629808
Shelton High School      83.441964
Thomas High School       84.254157
Wilson High School       84.021452
Wright High School       83.812757
Name: reading_score, dtype: float64


In [143]:
# Calculate the average reading score for 11th grade in each school
average_reading_11th_grade = eleventh_grade_students.groupby('school_name')['reading_score'].mean()

# Print
print("Average Reading Score for 11th Grade in Each School:")
print(average_reading_11th_grade)


Average Reading Score for 11th Grade in Each School:
school_name
Bailey High School       80.945643
Cabrera High School      83.788382
Figueroa High School     80.640339
Ford High School         80.403642
Griffin High School      84.288089
Hernandez High School    81.396140
Holden High School       83.815534
Huang High School        81.417476
Johnson High School      80.616027
Pena High School         84.335938
Rodriguez High School    80.864811
Shelton High School      84.373786
Thomas High School       83.585542
Wilson High School       83.764608
Wright High School       84.156322
Name: reading_score, dtype: float64


In [145]:
# Calculate the average reading score for 12th grade in each school
average_reading_12th_grade = twelfth_grade_students.groupby('school_name')['reading_score'].mean()

# Print
print("Average Reading Score for 12th Grade in Each School:")
print(average_reading_12th_grade)


Average Reading Score for 12th Grade in Each School:
school_name
Bailey High School       80.912451
Cabrera High School      84.287958
Figueroa High School     81.384863
Ford High School         80.662338
Griffin High School      84.013699
Hernandez High School    80.857143
Holden High School       84.698795
Huang High School        80.305983
Johnson High School      81.227564
Pena High School         84.591160
Rodriguez High School    80.376426
Shelton High School      82.781671
Thomas High School       83.831361
Wilson High School       84.317673
Wright High School       84.073171
Name: reading_score, dtype: float64


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [159]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 630, 645, float('inf')]
group_names = ["<$585", "$585-630", "$630-645", ">$645"]

In [171]:
# Create a new column to show budget per student in each row
school_data['spending_per_student'] = school_data['budget'] / school_data['size']

# Print the updated table to verify the new column
print(school_data[['school_name', 'spending_per_student']])


              school_name  spending_per_student
0       Huang High School                 655.0
1    Figueroa High School                 639.0
2     Shelton High School                 600.0
3   Hernandez High School                 652.0
4     Griffin High School                 625.0
5      Wilson High School                 578.0
6     Cabrera High School                 582.0
7      Bailey High School                 628.0
8      Holden High School                 581.0
9        Pena High School                 609.0
10     Wright High School                 583.0
11  Rodriguez High School                 637.0
12    Johnson High School                 650.0
13       Ford High School                 644.0
14     Thomas High School                 638.0


In [179]:
# Create a new column to define the spending ranges per student
school_data['spending_range'] = pd.cut(school_data['spending_per_student'], bins=spending_bins, labels=group_names)

# Print the updated table to verify the new column
print(school_data[['school_name', 'spending_per_student', 'spending_range']])


              school_name  spending_per_student spending_range
0       Huang High School                 655.0          >$645
1    Figueroa High School                 639.0       $630-645
2     Shelton High School                 600.0       $585-630
3   Hernandez High School                 652.0          >$645
4     Griffin High School                 625.0       $585-630
5      Wilson High School                 578.0          <$585
6     Cabrera High School                 582.0          <$585
7      Bailey High School                 628.0       $585-630
8      Holden High School                 581.0          <$585
9        Pena High School                 609.0       $585-630
10     Wright High School                 583.0          <$585
11  Rodriguez High School                 637.0       $630-645
12    Johnson High School                 650.0          >$645
13       Ford High School                 644.0       $630-645
14     Thomas High School                 638.0       $

In [219]:
# Merge student_data with school_data based on 'school_name'
merged_data = pd.merge(student_data[['school_name', 'math_score']], 
                       school_data[['school_name', 'spending_range']], 
                       on='school_name', how='left')

# Group by school_name and spending_range, then calculate the average math score
average_math_by_spending = merged_data.groupby(['school_name', 'spending_range'])['math_score'].mean().reset_index()

# Merge the average math score back into the school_data
school_data_with_avg_math = pd.merge(school_data, average_math_by_spending[['school_name', 'spending_range', 'math_score']], 
                                     on=['school_name', 'spending_range'], how='left')

# Renaming the column for clarity
school_data_with_avg_math = school_data_with_avg_math.rename(columns={'math_score': 'average_math_score_in_range'})

# Display the updated table with average math score within each spending range
display(school_data_with_avg_math[['school_name', 'spending_per_student', 'spending_range', 'average_math_score_in_range']])

/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_71113/2795779801.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  average_math_by_spending = merged_data.groupby(['school_name', 'spending_range'])['math_score'].mean().reset_index()


,school_name,spending_per_student,spending_range,average_math_score_in_range
0,Huang High School,655.0,>$645,76.629414
1,Figueroa High School,639.0,$630-645,76.711767
2,Shelton High School,600.0,$585-630,83.359455
3,Hernandez High School,652.0,>$645,77.289752
4,Griffin High School,625.0,$585-630,83.351499
5,Wilson High School,578.0,<$585,83.274201
6,Cabrera High School,582.0,<$585,83.061895
7,Bailey High School,628.0,$585-630,77.048432
8,Holden High School,581.0,<$585,83.803279
9,Pena High School,609.0,$585-630,83.839917


In [221]:
# Calculate the percentage passing rate for math in each spending range
merged_data['passing_math'] = merged_data['math_score'] >= 70

# Group by school_name and spending_range, calculate the passing rate
passing_math_rate = merged_data.groupby(['school_name', 'spending_range']).agg(
    total_students=('math_score', 'count'),
    passing_students=('passing_math', 'sum')
).reset_index()

# Calculate the passing rate as a percentage
passing_math_rate['passing_math_percentage'] = (passing_math_rate['passing_students'] / passing_math_rate['total_students']) * 100

# Merge the average math score and passing rate back into the school_data
school_data_with_avg_math_and_pass_rate = pd.merge(school_data, average_math_by_spending[['school_name', 'spending_range', 'math_score']], 
                                                   on=['school_name', 'spending_range'], how='left')

school_data_with_avg_math_and_pass_rate = pd.merge(school_data_with_avg_math_and_pass_rate, passing_math_rate[['school_name', 'spending_range', 'passing_math_percentage']], 
                                                   on=['school_name', 'spending_range'], how='left')

# Rename columns for clarity
school_data_with_avg_math_and_pass_rate = school_data_with_avg_math_and_pass_rate.rename(columns={
    'math_score': 'average_math_score_in_range',
    'passing_math_percentage': 'passing_math_percentage_in_range'
})

# Fill NaN values if necessary (in case some schools don't have students in certain ranges)
school_data_with_avg_math_and_pass_rate['passing_math_percentage_in_range'].fillna(0, inplace=True)

# Display the final table
display(school_data_with_avg_math_and_pass_rate[['school_name', 'spending_per_student', 'spending_range', 'average_math_score_in_range', 'passing_math_percentage_in_range']])




/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_71113/1143319963.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  passing_math_rate = merged_data.groupby(['school_name', 'spending_range']).agg(
/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_71113/1143319963.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sch

,school_name,spending_per_student,spending_range,average_math_score_in_range,passing_math_percentage_in_range
0,Huang High School,655.0,>$645,76.629414,65.683922
1,Figueroa High School,639.0,$630-645,76.711767,65.988471
2,Shelton High School,600.0,$585-630,83.359455,93.867121
3,Hernandez High School,652.0,>$645,77.289752,66.752967
4,Griffin High School,625.0,$585-630,83.351499,93.392371
5,Wilson High School,578.0,<$585,83.274201,93.867718
6,Cabrera High School,582.0,<$585,83.061895,94.133477
7,Bailey High School,628.0,$585-630,77.048432,66.680064
8,Holden High School,581.0,<$585,83.803279,92.505855
9,Pena High School,609.0,$585-630,83.839917,94.594595


In [227]:
# Define spending bins and labels
spending_bins = [0, 585, 630, 645, float('inf')]
group_names = ["<$585", "$585-630", "$630-645", ">$645"]

# Calculate spending per student
school_data['spending_per_student'] = school_data['budget'] / school_data['size']

# Create a new column 'spending_range' based on spending per student
school_data['spending_range'] = pd.cut(school_data['spending_per_student'], bins=spending_bins, labels=group_names)

# Merge student_data with school_data based on 'school_name'
merged_data = pd.merge(student_data[['school_name', 'math_score', 'reading_score']], 
                       school_data[['school_name', 'spending_range']], 
                       on='school_name', how='left')

# Group by school_name and spending_range, then calculate the average math score
average_math_by_spending = merged_data.groupby(['school_name', 'spending_range'])['math_score'].mean().reset_index()

# Calculate the percentage of students passing math (>= 70) in each spending range
# Create a new column 'passing_math' for math scores >= 70
merged_data['passing_math'] = merged_data['math_score'] >= 70

# Group by school_name and spending_range, calculate the passing rate for math
passing_math_rate = merged_data.groupby(['school_name', 'spending_range']).agg(
    total_students=('math_score', 'count'),
    passing_students=('passing_math', 'sum')
).reset_index()

# Calculate the passing rate for math as a percentage
passing_math_rate['passing_math_percentage'] = (passing_math_rate['passing_students'] / passing_math_rate['total_students']) * 100

# Calculate the percentage of students passing reading (>= 70) in each spending range
# Create a new column 'passing_reading' for reading scores >= 70
merged_data['passing_reading'] = merged_data['reading_score'] >= 70

# Group by school_name and spending_range, calculate the passing rate for reading
passing_reading_rate = merged_data.groupby(['school_name', 'spending_range']).agg(
    total_students=('reading_score', 'count'),
    passing_students=('passing_reading', 'sum')
).reset_index()

# Calculate the passing rate for reading as a percentage
passing_reading_rate['passing_reading_percentage'] = (passing_reading_rate['passing_students'] / passing_reading_rate['total_students']) * 100

# Merge the average math score, passing math rate, and passing reading rate back into the school_data
school_data_with_avg_math_and_pass_rate = pd.merge(school_data, average_math_by_spending[['school_name', 'spending_range', 'math_score']], 
                                                   on=['school_name', 'spending_range'], how='left')

school_data_with_avg_math_and_pass_rate = pd.merge(school_data_with_avg_math_and_pass_rate, passing_math_rate[['school_name', 'spending_range', 'passing_math_percentage']], 
                                                   on=['school_name', 'spending_range'], how='left')

school_data_with_avg_math_and_pass_rate = pd.merge(school_data_with_avg_math_and_pass_rate, passing_reading_rate[['school_name', 'spending_range', 'passing_reading_percentage']], 
                                                   on=['school_name', 'spending_range'], how='left')

# Rename columns for clarity
school_data_with_avg_math_and_pass_rate = school_data_with_avg_math_and_pass_rate.rename(columns={
    'math_score': 'average_math_score_in_range',
    'passing_math_percentage': 'passing_math_percentage_in_range',
    'passing_reading_percentage': 'passing_reading_percentage_in_range'
})


# Display the final table
display(school_data_with_avg_math_and_pass_rate[['school_name', 'spending_per_student', 'spending_range', 
                                                 'average_math_score_in_range', 
                                                 'passing_math_percentage_in_range', 
                                                 'passing_reading_percentage_in_range']])


/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_71113/4078231644.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  average_math_by_spending = merged_data.groupby(['school_name', 'spending_range'])['math_score'].mean().reset_index()
/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_71113/4078231644.py:24: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  passing_math_rate = merged_data.groupby(['school_name', 'spending_range']).agg(
/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_71113/4078231644.py:37: FutureWarning: The default of observed=False is deprecated and will be changed

,school_name,spending_per_student,spending_range,average_math_score_in_range,passing_math_percentage_in_range,passing_reading_percentage_in_range
0,Huang High School,655.0,>$645,76.629414,65.683922,81.316421
1,Figueroa High School,639.0,$630-645,76.711767,65.988471,80.739234
2,Shelton High School,600.0,$585-630,83.359455,93.867121,95.854628
3,Hernandez High School,652.0,>$645,77.289752,66.752967,80.862999
4,Griffin High School,625.0,$585-630,83.351499,93.392371,97.138965
5,Wilson High School,578.0,<$585,83.274201,93.867718,96.539641
6,Cabrera High School,582.0,<$585,83.061895,94.133477,97.039828
7,Bailey High School,628.0,$585-630,77.048432,66.680064,81.933280
8,Holden High School,581.0,<$585,83.803279,92.505855,96.252927
9,Pena High School,609.0,$585-630,83.839917,94.594595,95.945946


In [229]:
# Calculate the percentage overall passing rate in each spending range

# Define spending bins and labels
spending_bins = [0, 585, 630, 645, float('inf')]
group_names = ["<$585", "$585-630", "$630-645", ">$645"]

# Calculate spending per student
school_data['spending_per_student'] = school_data['budget'] / school_data['size']

# Create a new column 'spending_range' based on spending per student
school_data['spending_range'] = pd.cut(school_data['spending_per_student'], bins=spending_bins, labels=group_names)

# Merge student_data with school_data based on 'school_name'
merged_data = pd.merge(student_data[['school_name', 'math_score', 'reading_score']], 
                       school_data[['school_name', 'spending_range']], 
                       on='school_name', how='left')

# Group by school_name and spending_range, then calculate the average math score
average_math_by_spending = merged_data.groupby(['school_name', 'spending_range'])['math_score'].mean().reset_index()

# Calculate the percentage of students passing math (>= 70) in each spending range
# Create a new column 'passing_math' for math scores >= 70
merged_data['passing_math'] = merged_data['math_score'] >= 70

# Group by school_name and spending_range, calculate the passing rate for math
passing_math_rate = merged_data.groupby(['school_name', 'spending_range']).agg(
    total_students=('math_score', 'count'),
    passing_students=('passing_math', 'sum')
).reset_index()

# Calculate the passing rate for math as a percentage
passing_math_rate['passing_math_percentage'] = (passing_math_rate['passing_students'] / passing_math_rate['total_students']) * 100

# Calculate the percentage of students passing reading (>= 70) in each spending range
# Create a new column 'passing_reading' for reading scores >= 70
merged_data['passing_reading'] = merged_data['reading_score'] >= 70

# Group by school_name and spending_range, calculate the passing rate for reading
passing_reading_rate = merged_data.groupby(['school_name', 'spending_range']).agg(
    total_students=('reading_score', 'count'),
    passing_students=('passing_reading', 'sum')
).reset_index()

# Calculate the passing rate for reading as a percentage
passing_reading_rate['passing_reading_percentage'] = (passing_reading_rate['passing_students'] / passing_reading_rate['total_students']) * 100

# Merge the average math score, passing math rate, and passing reading rate back into the school_data
school_data_with_avg_math_and_pass_rate = pd.merge(school_data, average_math_by_spending[['school_name', 'spending_range', 'math_score']], 
                                                   on=['school_name', 'spending_range'], how='left')

school_data_with_avg_math_and_pass_rate = pd.merge(school_data_with_avg_math_and_pass_rate, passing_math_rate[['school_name', 'spending_range', 'passing_math_percentage']], 
                                                   on=['school_name', 'spending_range'], how='left')

school_data_with_avg_math_and_pass_rate = pd.merge(school_data_with_avg_math_and_pass_rate, passing_reading_rate[['school_name', 'spending_range', 'passing_reading_percentage']], 
                                                   on=['school_name', 'spending_range'], how='left')

# Rename columns for clarity
school_data_with_avg_math_and_pass_rate = school_data_with_avg_math_and_pass_rate.rename(columns={
    'math_score': 'average_math_score_in_range',
    'passing_math_percentage': 'passing_math_percentage_in_range',
    'passing_reading_percentage': 'passing_reading_percentage_in_range'
})

# Calculate overall passing rate as the average of math and reading passing rates
school_data_with_avg_math_and_pass_rate['overall_passing_rate_in_range'] = (
    (school_data_with_avg_math_and_pass_rate['passing_math_percentage_in_range'] +
     school_data_with_avg_math_and_pass_rate['passing_reading_percentage_in_range']) / 2
)

# Display the final table
display(school_data_with_avg_math_and_pass_rate[['school_name', 'spending_per_student', 'spending_range', 
                                                 'average_math_score_in_range', 
                                                 'passing_math_percentage_in_range', 
                                                 'passing_reading_percentage_in_range',
                                                 'overall_passing_rate_in_range']])


/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_71113/521757992.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  average_math_by_spending = merged_data.groupby(['school_name', 'spending_range'])['math_score'].mean().reset_index()
/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_71113/521757992.py:26: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  passing_math_rate = merged_data.groupby(['school_name', 'spending_range']).agg(
/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_71113/521757992.py:39: FutureWarning: The default of observed=False is deprecated and will be changed to

,school_name,spending_per_student,spending_range,average_math_score_in_range,passing_math_percentage_in_range,passing_reading_percentage_in_range,overall_passing_rate_in_range
0,Huang High School,655.0,>$645,76.629414,65.683922,81.316421,73.500171
1,Figueroa High School,639.0,$630-645,76.711767,65.988471,80.739234,73.363852
2,Shelton High School,600.0,$585-630,83.359455,93.867121,95.854628,94.860875
3,Hernandez High School,652.0,>$645,77.289752,66.752967,80.862999,73.807983
4,Griffin High School,625.0,$585-630,83.351499,93.392371,97.138965,95.265668
5,Wilson High School,578.0,<$585,83.274201,93.867718,96.539641,95.203679
6,Cabrera High School,582.0,<$585,83.061895,94.133477,97.039828,95.586652
7,Bailey High School,628.0,$585-630,77.048432,66.680064,81.933280,74.306672
8,Holden High School,581.0,<$585,83.803279,92.505855,96.252927,94.379391
9,Pena High School,609.0,$585-630,83.839917,94.594595,95.945946,95.270270


### Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [ ]:
# Create a new column for the bin groups


Look for the total count of test scores that pass 70% or higher




In [ ]:
# math_pass_size




In [ ]:
# read_pass_size




In [ ]:
# Calculate the overall passing rate for different school size




### Scores by School Type

* Perform the same operations as above, based on school type.

In [ ]:
# Create bins and groups, school type {'Charter', 'District'}




Find counts of the passing 70 or higher score for the both test


In [ ]:
# math pass size




In [ ]:
# reading pass size



In [ ]:
# Calculate the overall passing rate

